### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [5]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()



os.environ['groq_api_key']=os.getenv("GROQ_API_KEY")


In [ ]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b")
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f47d4ecbd10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f47d4d2c950>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [10]:
from langchain_core.messages import HumanMessage, SystemMessage
messages=[
    SystemMessage(content="You are a helpful assistant that translates English to Hindi."),
    HumanMessage(content="Hello, how are you?")
]
result=model.invoke(messages)
print(result.content)

नमस्ते, आप कैसे हैं?


In [17]:
result

AIMessage(content='Bonjour, comment allez-vous ? \n', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 26, 'total_tokens': 37, 'completion_time': 0.022, 'prompt_time': 0.003249601, 'queue_time': None, 'total_time': 0.025249601}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_e238d2cf52', 'finish_reason': 'stop', 'logprobs': None}, id='run-88f30adf-3746-488d-ad12-13bf16f1d259-0', usage_metadata={'input_tokens': 26, 'output_tokens': 11, 'total_tokens': 37})

In [12]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
parsed_output = parser.parse(result.content)
print(parsed_output)

नमस्ते, आप कैसे हैं?


In [13]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'नमस्ते, आप कैसे हैं?'

In [23]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant that translates English to Hindi."),
    ("human","{text_to_translate}")
])




In [20]:
result=prompt.invoke({"text_to_translate":"Hello"})

In [21]:
result.to_messages()

[SystemMessage(content='You are a helpful assistant that translates the the following to Hindi.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='{text_to_translate}', additional_kwargs={}, response_metadata={})]

In [24]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"text_to_translate":"Hello"})

'नमस्ते।'